In [1]:
import xgboost
import tqdm
import pandas as pd
import ast 
import os.path
import os
pd.set_option('plotting.backend', 'pandas_bokeh')
import pandas_bokeh
pandas_bokeh.output_notebook()
import joblib
import sklearn
import warnings
warnings.filterwarnings('ignore')
import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler
import json

Loading BokehJS ...

In [2]:
tokens_list_df = pd.read_csv('./data_records/tokens_list_df.csv')
tokens_list = tokens_list_df['0'].tolist()

In [3]:
def objective(trial: Trial,X,Y) -> float:
    
    joblib.dump(study, 'study.pkl')
    
    train_X,test_X,train_y,test_y = sklearn.model_selection.train_test_split(X, Y, test_size = 0.30,random_state = 101)

    param = {"n_estimators" : trial.suggest_int('n_estimators', 0, 1000),
            'max_depth':trial.suggest_int('max_depth', 2, 25),
            'reg_alpha':trial.suggest_int('reg_alpha', 0, 5),
            'reg_lambda':trial.suggest_int('reg_lambda', 0, 5),
            'min_child_weight':trial.suggest_int('min_child_weight', 0, 5),
            'gamma':trial.suggest_int('gamma', 0, 5),
            'learning_rate':trial.suggest_loguniform('learning_rate',0.005,0.5),
            'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.1,1,0.01),
            'nthread' : -1,
            'tree_method':'hist'}
    model = xgboost.XGBRegressor(**param)

    model.fit(train_X,train_y)

    return sklearn.model_selection.cross_val_score(model,test_X,test_y).mean()

In [4]:
globals()[f'pnls_df_{1346049}'] = pd.read_excel(f'./data_records_13_4_2021/instwise/1346049.xlsx',parse_dates=['last_trade_time'])
del globals()[f'pnls_df_{1346049}']['Unnamed: 0']
del globals()[f'pnls_df_{1346049}']['Unnamed: 0.1']

globals()[f'pnls_df_{1346049}'] = globals()[f'pnls_df_{1346049}'].replace({'\'': '"'}, regex=True)
ohlc_df = pd.io.json.json_normalize(globals()[f'pnls_df_{1346049}'].ohlc.apply(json.loads))

globals()[f'pnls_df_OHLC{1346049}'] = pd.concat([globals()[f'pnls_df_{1346049}'],ohlc_df],axis=1)
del_column_list = ['tradable', 'mode', 'instrument_token', 'ohlc','timestamp', 'depth','last_trade_time']
globals()[f'pnls_df_OHLC{1346049}'].drop(del_column_list, axis = 1, inplace = True)

x=globals()[f'pnls_df_OHLC{1346049}'].drop([len(globals()[f'pnls_df_OHLC{1346049}'].index)-1]) 

y = globals()[f'pnls_df_OHLC{1346049}']['last_price'][1:].to_numpy()

In [ ]:
# for one_key in x.keys():
#     print('-'*55)
#     print(f'Training for {one_key}')
    
#     locals()[f'y_{one_key}'] = locals()[f'pnls_df_OHLC{1346049}'][one_key][1:].to_numpy()
    
#     file_name = f'./xgb_models_tick_data/xgbmodel_{one_key}'
#     if(os.path.isfile(file_name)):
#         continue
    
#     study = optuna.create_study(direction='maximize',sampler=TPESampler())
#     study.optimize(lambda trial : objective(trial,x,globals()[f'y_{one_key}']),n_trials= 100)
#     print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

#     locals()[f'train_params_{one_key}'] = study.best_trial.params
#     locals()[f'train_params_{one_key}']['nthread'] = -1
#     locals()[f'train_params_{one_key}']['tree_method'] = 'hist'
    
#     this_best_params_df = pd.DataFrame(locals()[f'train_params_{one_key}'],index=[0])
#     this_best_params_df.to_excel(f'./xgb_models_tick_data/this_best_params_df_{one_key}.xlsx')
    
#     locals()[f'xgbmodel_{one_key}']  = xgboost.XGBRegressor(**locals()[f'train_params_{one_key}'])
#     locals()[f'xgbmodel_{one_key}'].fit(x,locals()[f'y_{one_key}'])
#     locals()[f'xgbmodel_{one_key}'].save_model(f'./xgb_models_tick_data/xgbmodel_{one_key}')

In [ ]:
tkn_counter = 1
for one_tkn in tqdm.tqdm(tokens_list):
    print('*'*55)
    print(f'Training for TKN : {one_tkn} : {tkn_counter}')
    tkn_counter = tkn_counter+1
    file_read_name = f'./data_records/instwise/{one_tkn}.xlsx'
    
    if(not os.path.isfile(file_read_name)):
        continue
    
    locals()[f'pnls_df_{one_tkn}'] = pd.read_excel(file_read_name)
    
#     del locals()[f'pnls_df_{one_tkn}']['Unnamed: 0']
#     del locals()[f'pnls_df_{one_tkn}']['Unnamed: 0.1']

    locals()[f'pnls_df_{one_tkn}'] = locals()[f'pnls_df_{one_tkn}'].replace({'\'': '"'}, regex=True)
    ohlc_df = pd.io.json.json_normalize(locals()[f'pnls_df_{one_tkn}'].ohlc.apply(json.loads))

    locals()[f'pnls_df_OHLC{one_tkn}'] = pd.concat([locals()[f'pnls_df_{one_tkn}'],ohlc_df],axis=1)
    del_column_list = ['tradable', 'mode', 'instrument_token', 'ohlc','timestamp', 'depth','last_trade_time']
    locals()[f'pnls_df_OHLC{one_tkn}'].drop(del_column_list, axis = 1, inplace = True)

    x=locals()[f'pnls_df_OHLC{one_tkn}'].drop([len(locals()[f'pnls_df_OHLC{one_tkn}'].index)-1]) 
    
    for one_key in x.keys():
        locals()[f'xgb_model_{one_key}'] = xgboost.Booster({'nthread':4})
        locals()[f'xgb_model_{one_key}'].load_model(f"./xgb_models_tick_data/xgbmodel_{one_key}")
        print(f'   for key : {one_key}')
        one_para = pd.read_excel(f'./xgb_models_tick_data/this_best_params_df_{one_key}.xlsx')
        del one_para['Unnamed: 0']
        one_para_dict = one_para.to_dict('index')[0]
        update_para = {'process_type': 'default','refresh_leaf': True}
        one_para = {**one_para_dict, **update_para}
        
        del one_para['n_estimators']
        locals()[f'y_{one_key}'] = globals()[f'pnls_df_OHLC{one_tkn}'][one_key][1:].to_numpy()
        locals()[f'xgb_model_{one_key}'] = xgboost.train(one_para, 
                          dtrain=xgboost.DMatrix(x, locals()[f'y_{one_key}']), 
                          xgb_model=globals()[f'xgb_model_{one_key}'])
        
        locals()[f'xgb_model_{one_key}'].save_model(f'./xgb_models_tick_data/xgbmodel_{one_key}')

In [ ]:
for i in rangeege(4):
    
    tkn_counter = 1
    for one_tkn in tqdm.tqdm(tokens_list):
        print('*'*55)
        print(f'Training for TKN : {one_tkn} : {tkn_counter}')
        tkn_counter = tkn_counter+1
        file_read_name = f'./data_records/instwise/{one_tkn}.xlsx'

        if(not os.path.isfile(file_read_name)):
            continue

        locals()[f'pnls_df_{one_tkn}'] = pd.read_excel(file_read_name)

    #     del locals()[f'pnls_df_{one_tkn}']['Unnamed: 0']
    #     del locals()[f'pnls_df_{one_tkn}']['Unnamed: 0.1']

        locals()[f'pnls_df_{one_tkn}'] = locals()[f'pnls_df_{one_tkn}'].replace({'\'': '"'}, regex=True)
        ohlc_df = pd.io.json.json_normalize(locals()[f'pnls_df_{one_tkn}'].ohlc.apply(json.loads))

        locals()[f'pnls_df_OHLC{one_tkn}'] = pd.concat([locals()[f'pnls_df_{one_tkn}'],ohlc_df],axis=1)
        del_column_list = ['tradable', 'mode', 'instrument_token', 'ohlc','timestamp', 'depth','last_trade_time']
        locals()[f'pnls_df_OHLC{one_tkn}'].drop(del_column_list, axis = 1, inplace = True)

        x=locals()[f'pnls_df_OHLC{one_tkn}'].drop([len(locals()[f'pnls_df_OHLC{one_tkn}'].index)-1]) 

        for one_key in x.keys():
            locals()[f'xgb_model_{one_key}'] = xgboost.Booster({'nthread':4})
            locals()[f'xgb_model_{one_key}'].load_model(f"./xgb_models_tick_data/xgbmodel_{one_key}")
            print(f'   for key : {one_key}')
            one_para = pd.read_excel(f'./xgb_models_tick_data/this_best_params_df_{one_key}.xlsx')
            del one_para['Unnamed: 0']
            one_para_dict = one_para.to_dict('index')[0]
            update_para = {'process_type': 'default','refresh_leaf': True}
            one_para = {**one_para_dict, **update_para}

            del one_para['n_estimators']
            locals()[f'y_{one_key}'] = globals()[f'pnls_df_OHLC{one_tkn}'][one_key][1:].to_numpy()
            locals()[f'xgb_model_{one_key}'] = xgboost.train(one_para, 
                              dtrain=xgboost.DMatrix(x, locals()[f'y_{one_key}']), 
                              xgb_model=globals()[f'xgb_model_{one_key}'])

            locals()[f'xgb_model_{one_key}'].save_model(f'./xgb_models_tick_data/xgbmodel_{one_key}') 
    
    
    
    tkn_counter = 1
    for one_tkn in tqdm.tqdm(tokens_list):
        print('*'*55)
        print(f'Training for TKN : {one_tkn} : {tkn_counter}')
        tkn_counter = tkn_counter+1
        file_read_name =./data_records_13_4_2021/13_4_2021/ds_13_4_2021/instwise/{one_tkn}.xlsx'

        if(not os.path.isfile(file_read_name)):
            continue

        locals()[f'pnls_df_{one_tkn}'] = pd.read_excel(file_read_name)

        del locals()[f'pnls_df_{one_tkn}']['Unnamed: 0']
        del locals()[f'pnls_df_{one_tkn}']['Unnamed: 0.1']

        locals()[f'pnls_df_{one_tkn}'] = locals()[f'pnls_df_{one_tkn}'].replace({'\'': '"'}, regex=True)
        ohlc_df = pd.io.json.json_normalize(locals()[f'pnls_df_{one_tkn}'].ohlc.apply(json.loads))

        locals()[f'pnls_df_OHLC{one_tkn}'] = pd.concat([locals()[f'pnls_df_{one_tkn}'],ohlc_df],axis=1)
        del_column_list = ['tradable', 'mode', 'instrument_token', 'ohlc','timestamp', 'depth','last_trade_time']
        locals()[f'pnls_df_OHLC{one_tkn}'].drop(del_column_list, axis = 1, inplace = True)

        x=locals()[f'pnls_df_OHLC{one_tkn}'].drop([len(locals()[f'pnls_df_OHLC{one_tkn}'].index)-1]) 

        for one_key in x.keys():
            locals()[f'xgb_model_{one_key}'] = xgboost.Booster({'nthread':4})
            locals()[f'xgb_model_{one_key}'].load_model(f"./xgb_models_tick_data/xgbmodel_{one_key}")
            print(f'   for key : {one_key}')
            one_para = pd.read_excel(f'./xgb_models_tick_data/this_best_params_df_{one_key}.xlsx')
            del one_para['Unnamed: 0']
            one_para_dict = one_para.to_dict('index')[0]
            update_para = {'process_type': 'default','refresh_leaf': True}
            one_para = {**one_para_dict, **update_para}

            del one_para['n_estimators']
            locals()[f'y_{one_key}'] = globals()[f'pnls_df_OHLC{one_tkn}'][one_key][1:].to_numpy()
            locals()[f'xgb_model_{one_key}'] = xgboost.train(one_para, 
                              dtrain=xgboost.DMatrix(x, locals()[f'y_{one_key}']), 
                              xgb_model=globals()[f'xgb_model_{one_key}'])

            locals()[f'xgb_model_{one_key}'].save_model(f'./xgb_models_tick_data/xgbmodel_{one_key}') 
    
    
    
    
    

In [5]:
future_window = 30
initial_dmat = xgboost.DMatrix(x[:1].values)
all_future_json_list_for_df = []

In [6]:
for one_future in range(future_window):
    this_future_dict = {}
    print('-'*55)
    print(f'######  Future number : {one_future} #######')

    for one_key in x.keys():
        globals()[f'xgb_model_{one_key}'] = xgboost.Booster({'nthread':4})
        globals()[f'xgb_model_{one_key}'].load_model(f"./xgb_models_tick_data/xgbmodel_{one_key}")
        locals()[one_key] = globals()[f'xgb_model_{one_key}'].predict(initial_dmat)
#         print(f'Predicted {one_key} : {locals()[one_key][0]}')
        this_future_dict[f'{one_key}'] = locals()[one_key][0]
            
#     print(this_future_dict)
    this_future_dict_df = pd.DataFrame(this_future_dict,index=[0])
    initial_dmat = xgboost.DMatrix(this_future_dict_df)
    all_future_json_list_for_df.append(this_future_dict)
    
all_future_df = pd.DataFrame(all_future_json_list_for_df)
all_future_df

-------------------------------------------------------
######  Future number : 0 #######
-------------------------------------------------------
######  Future number : 1 #######
-------------------------------------------------------
######  Future number : 2 #######
-------------------------------------------------------
######  Future number : 3 #######
-------------------------------------------------------
######  Future number : 4 #######
-------------------------------------------------------
######  Future number : 5 #######
-------------------------------------------------------
######  Future number : 6 #######
-------------------------------------------------------
######  Future number : 7 #######
-------------------------------------------------------
######  Future number : 8 #######
-------------------------------------------------------
######  Future number : 9 #######
-------------------------------------------------------
######  Future number : 10 #######
---------

,last_price,last_quantity,average_price,volume,buy_quantity,sell_quantity,change,oi,oi_day_high,oi_day_low,open,high,low,close
0,1816.485107,57.569672,7.363746e+07,314419072.0,-16449.578125,186760.484375,-21.332952,-564.909241,-135.322327,-1561.769287,2097.895996,390.813141,30.190094,373.73941
1,1913.340698,71.195473,1.595248e+07,207377776.0,-948.697754,299327.312500,-24.665676,-919.451294,-44.441879,2516.954834,2062.648926,-3390.158691,265.005890,373.73941
2,1914.494019,715.969299,8.424998e+06,265690848.0,23801.148438,342228.718750,-37.920193,-1313.313354,-4255.713379,10694.279297,2059.400146,-1055.480957,-182.269440,373.73941
3,1914.911987,652.632935,-6.046951e+07,160671456.0,83986.671875,426172.656250,-11.928080,-1325.533447,-3710.478271,11782.777344,2066.801025,-5144.340820,-137.536819,373.73941
4,1856.286499,635.785645,2.808407e+06,210989984.0,145676.234375,482335.593750,-25.437271,-1958.645142,-3479.791992,30814.396484,2033.564331,-382.427246,-209.502625,373.73941
5,1915.342651,879.765198,-1.334331e+07,171629856.0,216942.937500,608640.125000,-13.515894,-1180.619995,-3515.385498,8301.915039,2045.789917,-4573.145996,-219.107620,373.73941
6,1856.191772,579.299805,1.453884e+07,227475616.0,218734.687500,589379.937500,-37.869518,-2218.303223,-3373.118408,19958.339844,2033.606323,-1263.097046,-166.907410,373.73941
7,1915.136597,735.898132,1.156647e+06,189568768.0,266656.281250,610680.687500,-30.201790,-1155.470947,-3410.431152,-9592.418945,2046.406372,-4158.181152,-289.130066,373.73941
8,1917.086792,907.906677,-9.203951e+05,182481664.0,320933.968750,614079.187500,-30.011307,-1178.545532,-3428.309326,-1595.852295,2046.965942,-5159.833984,-262.818054,373.73941
9,1856.221069,620.798096,-3.476800e+06,227355936.0,301389.218750,525394.062500,-38.339783,-2207.943359,-3400.871582,24259.140625,2031.107300,-1356.616699,-153.884949,373.73941


In [7]:
for one_key in x.keys():
    plt = all_future_df[one_key].plot()
#     plt.plot()